In [32]:
import numpy as np
import copy as cp
import csv
import time as tp
import pandas as pd
import queue

In [33]:
class Sudoku:
    
    board = None
    size = None
    
    def __init__(self,file = None, board = None, size = None):
        
        if file:
            self.board = np.zeros((9,9))
            self.size = 9
            with open(file) as csv_file:
                csv_reader = csv.reader(csv_file,delimiter = ',')
                
                for i,row in enumerate(csv_reader):
                    for j,number in enumerate(row[1:]):
                        if number != '':
                            self.board[i,j] = number
        else:
            self.board = board
            self.size = size


    def printBoard(self):
        print "\n"
        for i in range(0,9):
            print self.board[i,0:3], "|",self.board[i,3:6],"|",self.board[i,6:9]
            if (i+1)%3 == 0 and i!= 8:
                print "------------------------------------"
        print "\n"
        
        
    def getRange(self,cuadrant):
        
        row,col = [],[]
        
        if cuadrant <3 :
            row = range(0,3)
            
        if cuadrant >= 3 and cuadrant< 6:
            row = range(3,6)
            
        if cuadrant >= 6 and cuadrant < 9:
            row = range(6,9)

        if cuadrant in (0,3,6):
            col = range(0,3)
        
        if cuadrant in (1,4,7):
            col = range(3,6)
                    
        if cuadrant in (2,5,8):
            col = range(6,9)
        
        return row,col
        
    def getCuadrant(self,pos):
        
        row,col =  pos
        
        if row <3:
            cuad = 0
        
        if row >=3 and row <6:
            cuad = 3
        
        if row >=6:
            cuad = 6
        
        cuad += col/3
        
        return cuad
    
    def getCuadrantMoves(self,nums):
        rows,cols = nums

        moves = []
        for i in rows:
            for j in cols:
                moves.append((i,j))
                
        return moves
        
    
    def checkCuadrant(self,pos):
        cuad = self.getCuadrant(pos)        
        rows,cols = self.getRange(cuad)
        
        if self.board[pos] == 0:
            return True
            
        
        for i in rows:
            for j in cols:
                if (i,j) != pos and self.board[i,j] == self.board[pos]:
                    return False
        
        return True
    
    def getRowsandCols(self,pos):
        row,col = pos
        moves = []
        for i in range(self.size):
            if i != row and not (i,col) in moves:
                moves.append((i,col))
            if i != col and not (row,i) in moves:
                moves.append((row,i))
        
        return moves
        
    def checkRowandCol(self,pos):
        row,col = pos
        
        if self.board[pos]==0:
            return True
        
        moves = self.getRowsandCols(pos)
        
        for mov in moves:
            if self.board[mov] == self.board[row,col]:
                return False            
        return True    
    
    #Funcion que verifica que la posicion no tenga conflictos
    def isOk(self,pos):
        
        if not self.checkCuadrant(pos) or not self.checkRowandCol(pos):
            return False
        return True
    
    def isValid(self):

        for i in range(self.size):
            for j in range(self.size):
                if not self.isOk((i,j)):
                    return False
        return True
    
    #Funcion que verifica que el tablero este resuelto
    def isGoal(self): 
     
        for i in range(self.size):
            for j in range(self.size):
                if not self.isOk((i,j)) or self.board[i,j] == 0:
                    return False
        return True

In [34]:
#board = Sudoku("solver.csv")
board = Sudoku("Sudoku.csv")
board.printBoard()


#board.isValid()
# 1,4,7  col (3,6)
# 2,5,8  col (6,9)
# 0 ,3,6 col (0,3)

# < 3 ro



[0. 1. 0.] | [7. 9. 0.] | [5. 4. 0.]
[0. 0. 0.] | [0. 6. 0.] | [0. 0. 0.]
[0. 6. 0.] | [0. 0. 4.] | [0. 9. 0.]
------------------------------------
[2. 0. 0.] | [5. 0. 0.] | [0. 0. 1.]
[0. 0. 9.] | [0. 0. 0.] | [0. 0. 0.]
[7. 0. 0.] | [0. 0. 0.] | [8. 2. 5.]
------------------------------------
[0. 0. 0.] | [0. 0. 0.] | [0. 1. 0.]
[0. 4. 1.] | [0. 0. 2.] | [6. 0. 0.]
[5. 0. 0.] | [3. 0. 0.] | [0. 0. 0.]




In [35]:
class Back(Sudoku):
    board = None
    neighbours = None
    domains = None
    variables = None
    
    def __init__(self,file = None, board = None, size = None):
        
        Sudoku.__init__(self,file,board,size)
        self.domains = {}
        constraints = {}
        
        self.variables = self.getEmpty()
        
        for i in self.variables:
                    
            moves = self.getRowsandCols(i)
            cuad = self.getCuadrant(i)
                    

            moves += self.getCuadrantMoves((self.getRange(cuad)))
                    
            value = self.board[i]
            constraints[i] = moves

                
        for var in self.variables:
            if self.board[var] == 0:

                posdomain = range(1,self.size+1)
                self.domains[var] = []
                for num in posdomain:
                    
                    find = False
                    
                    for i in constraints[var]:
                        if self.board[i] == num:
                            find = True
    
                    if not find:                    
                        self.domains[var].append(num)

        if len(self.variables) > 0:
            self.getNeighbours(self.variables[0])
        else:
            self.neighbours  = []

    
        
    
    def getEmpty(self):
        empty = []
        for i in range(self.size):
            for j in range(self.size):
                if self.board[i,j] == 0:
                    empty.append((i,j))
        return empty
    
    def createNeighbour(self,pos,value):
        
        board = self.board.copy()
        board[pos] = value
        return (board,self.size)
    
    
        
    
    def getNeighbours(self,actual):
        self.neighbours = []
        values = range(1,10)
        
        for i in self.domains[actual]:
            self.neighbours.append(self.createNeighbour(actual,i))

#file = "solver.csv"
#file = "Sudoku.csv"
#file = "easy.csv"

#backtracking.printBoard()

In [36]:
def execBack(node):
    
    if node.isGoal():
        return node
    
    if not node.isValid():
        return False


    
    for vec in node.neighbours:
    
        ans =  execBack(Back(None,vec[0],vec[1]))
        if ans:
            return ans
    
    return False
    
file = "test.csv"
backtracking = Back(file)    
ans = execBack(backtracking)
ans.printBoard()



[4. 8. 3.] | [9. 2. 1.] | [6. 5. 7.]
[9. 6. 7.] | [3. 4. 5.] | [8. 2. 1.]
[2. 5. 1.] | [8. 7. 6.] | [4. 9. 3.]
------------------------------------
[5. 4. 8.] | [1. 3. 2.] | [9. 7. 6.]
[7. 2. 9.] | [5. 6. 4.] | [1. 3. 8.]
[1. 3. 6.] | [7. 9. 8.] | [2. 4. 5.]
------------------------------------
[3. 7. 2.] | [6. 8. 9.] | [5. 1. 4.]
[8. 1. 4.] | [2. 5. 3.] | [7. 6. 9.]
[6. 9. 5.] | [4. 1. 7.] | [3. 8. 2.]




In [37]:
class Ac3(Sudoku):
    domains = None
    constraints = None
    variables = None
    asig = None
    nextV = None
    
    def __init__(self,file = None, board = None, size = None,solved = False,domains = None,variables = None):
        
        Sudoku.__init__(self,file,board,size)
        
        if solved:
            self.variables = variables
            self.domains = domains
            return
        
        self.variables = []
        self.domains = {}
        self.constraints = [] # Posiciones que se afectan entre si
        self.places = {} #Direct con las posiciones que afectan a una variable en especifico
        self.asig = {}  #Valores ya asignados
        
        #Creamos un arreglo con las variables
        for i in range(self.size):
            for j in range(self.size):
                self.variables.append((i,j))
                
        
        for i in self.variables:
            if self.board[i] != 0:
                self.asig[i] = [self.board[i]]
                self.domains[i] = [self.board[i]]
                
            else:
                self.domains[i] = range(1,10)
                self.asig[i] = 0
        
        #Llenamos el direct de posiciones que afectan a una variable
        for i in self.variables:
            self.places[i] = []
            
            moves = sorted(self.getArea(i), key = lambda ax: ax[0])
            index = moves.index(i)
            moves.pop(index)
            
            self.places[i] = moves
            
        for i in self.variables:
            for j in self.places[i]:
                self.constraints.append((i,j))
                
		


    def getArea(self,pos):
        moves = self.getRowsandCols(pos)
        cuad = self.getCuadrant(pos)
        for i in self.getCuadrantMoves((self.getRange(cuad))):
            if not i in moves:
                moves.append(i)
        
        return moves
    
    def esConsistente(self,x,var1,var2):
        
        for y in self.domains[var2]:
            if var2 in self.places[var1] and y!= x:
                return True
        return False
    
    def arcReduce(self,var1,var2):
        
        flag = False
        
        for i in self.domains[var1]:
            if not self.esConsistente(i,var1,var2):
                index = self.domains[var1].index(i)
                self.domains[var1].pop(index)

                flag = True
        return flag
    
    def getNeighbours(self,var):
        
        vecs = []
        vari = self.variables[var]
        
        for i in self.domains[vari]:
            board = self.board.copy()
            board[vari] = i
            #
            vecs.append(board)
        
        band = True
        while band:
            var += 1
            
            if var < len(self.variables):
                vari = self.variables[var]

            
                if self.board[vari] == 0:
                    band = False
            else:
                
                band = False
                
        self.nextV = var
        return vecs
    
       

In [38]:
def execAC3(node):
    
    cola = []
    
    for evaluate in node.constraints:
        cola.append(evaluate)
    
    while cola:
        
        var1,var2 = cola.pop(0)
        
        if node.arcReduce(var1,var2):
            if len(node.domains[var1]) == 0:
                return False
            
            for news in node.places[var1]:

                if news != var1:

                    cola.append((news,var1))

    return True
    
file = "test.csv"
p = Ac3(file) 
execAC3(p)



True

In [39]:
def execBackAc3(node,actual):
    
    if not node.isValid() or len(node.variables) < actual:
        return False    
    
    if node.isGoal():
        
        node.printBoard()
        return True
    

    
    for i in node.getNeighbours(actual):

        new_node = Ac3(None,i,node.size,domains=node.domains,variables=node.variables,solved=True)
        if execBackAc3(new_node,node.nextV):
            return True
    
    return False
        
        #print i
    
execBackAc3(p,0)



[4. 8. 3.] | [9. 2. 1.] | [6. 5. 7.]
[9. 6. 7.] | [3. 4. 5.] | [8. 2. 1.]
[2. 5. 1.] | [8. 7. 6.] | [4. 9. 3.]
------------------------------------
[5. 4. 8.] | [1. 3. 2.] | [9. 7. 6.]
[7. 2. 9.] | [5. 6. 4.] | [1. 3. 8.]
[1. 3. 6.] | [7. 9. 8.] | [2. 4. 5.]
------------------------------------
[3. 7. 2.] | [6. 8. 9.] | [5. 1. 4.]
[8. 1. 4.] | [2. 5. 3.] | [7. 6. 9.]
[6. 9. 5.] | [4. 1. 7.] | [3. 8. 2.]




True

In [40]:
def execAll(file):
    
    back = Back(file) 
    print ("\n---------------- Tablero a resolver ----------------")
    back.printBoard()
    
    print("\n---------------- BackTracking ---------------------")
    prev = tp.time()
    res = execBack(back)
    res.printBoard()
    backt = tp.time() - prev
    print"------ Resuelto en: ",backt,"segundos ------"
    
    
    
    print("\n---------------- AC3 ---------------------")
    ac = Ac3(file)
    prev = tp.time()
    execAC3(ac)
    execBackAc3(ac,0)
    act = tp.time() - prev
    print"------ Resuelto en: ",act,"segundos ------"    
    
    return backt,act

bt = []
act = []
btn,actn = execAll("test.csv")
bt.append(btn)
act.append(actn)


---------------- Tablero a resolver ----------------


[0. 0. 3.] | [0. 2. 0.] | [6. 0. 0.]
[9. 0. 0.] | [3. 0. 5.] | [0. 0. 1.]
[0. 0. 1.] | [8. 0. 6.] | [4. 0. 0.]
------------------------------------
[0. 0. 8.] | [1. 0. 2.] | [9. 0. 0.]
[7. 0. 0.] | [0. 0. 0.] | [0. 0. 8.]
[0. 0. 6.] | [7. 0. 8.] | [2. 0. 0.]
------------------------------------
[0. 0. 2.] | [6. 0. 9.] | [5. 0. 0.]
[8. 0. 0.] | [2. 0. 3.] | [0. 0. 9.]
[0. 0. 5.] | [0. 1. 0.] | [3. 0. 0.]



---------------- BackTracking ---------------------


[4. 8. 3.] | [9. 2. 1.] | [6. 5. 7.]
[9. 6. 7.] | [3. 4. 5.] | [8. 2. 1.]
[2. 5. 1.] | [8. 7. 6.] | [4. 9. 3.]
------------------------------------
[5. 4. 8.] | [1. 3. 2.] | [9. 7. 6.]
[7. 2. 9.] | [5. 6. 4.] | [1. 3. 8.]
[1. 3. 6.] | [7. 9. 8.] | [2. 4. 5.]
------------------------------------
[3. 7. 2.] | [6. 8. 9.] | [5. 1. 4.]
[8. 1. 4.] | [2. 5. 3.] | [7. 6. 9.]
[6. 9. 5.] | [4. 1. 7.] | [3. 8. 2.]


------ Resuelto en:  1.33014798164 segundos ------

---------------- AC

In [41]:
btn,actn = execAll("easy.csv")
bt.append(btn)
act.append(actn)


---------------- Tablero a resolver ----------------


[0. 4. 0.] | [2. 5. 0.] | [0. 0. 9.]
[2. 9. 0.] | [0. 0. 8.] | [0. 3. 0.]
[0. 0. 0.] | [0. 0. 0.] | [0. 0. 0.]
------------------------------------
[0. 0. 0.] | [0. 7. 0.] | [1. 9. 0.]
[3. 7. 0.] | [9. 0. 5.] | [0. 0. 2.]
[6. 0. 0.] | [0. 0. 4.] | [0. 0. 0.]
------------------------------------
[0. 5. 4.] | [1. 0. 0.] | [6. 2. 0.]
[0. 0. 0.] | [0. 0. 0.] | [0. 0. 0.]
[0. 6. 0.] | [0. 0. 0.] | [5. 0. 0.]



---------------- BackTracking ---------------------


[7. 4. 6.] | [2. 5. 3.] | [8. 1. 9.]
[2. 9. 1.] | [6. 4. 8.] | [7. 3. 5.]
[5. 8. 3.] | [7. 9. 1.] | [2. 4. 6.]
------------------------------------
[4. 2. 5.] | [3. 7. 6.] | [1. 9. 8.]
[3. 7. 8.] | [9. 1. 5.] | [4. 6. 2.]
[6. 1. 9.] | [8. 2. 4.] | [3. 5. 7.]
------------------------------------
[9. 5. 4.] | [1. 8. 7.] | [6. 2. 3.]
[1. 3. 7.] | [5. 6. 2.] | [9. 8. 4.]
[8. 6. 2.] | [4. 3. 9.] | [5. 7. 1.]


------ Resuelto en:  85.5635590553 segundos ------

---------------- AC

In [42]:
btn,actn = execAll("medium.csv")
bt.append(btn)
act.append(actn)


---------------- Tablero a resolver ----------------


[2. 5. 0.] | [0. 9. 0.] | [0. 0. 3.]
[6. 0. 0.] | [0. 0. 0.] | [0. 0. 0.]
[0. 0. 8.] | [0. 5. 4.] | [0. 0. 0.]
------------------------------------
[0. 0. 9.] | [0. 8. 7.] | [0. 0. 0.]
[0. 0. 0.] | [0. 0. 0.] | [1. 0. 6.]
[0. 0. 4.] | [0. 2. 0.] | [5. 0. 0.]
------------------------------------
[0. 0. 0.] | [0. 0. 0.] | [0. 9. 4.]
[0. 8. 0.] | [0. 0. 0.] | [0. 0. 0.]
[0. 1. 7.] | [0. 0. 2.] | [0. 0. 0.]



---------------- BackTracking ---------------------


[2. 5. 1.] | [6. 9. 8.] | [7. 4. 3.]
[6. 4. 3.] | [2. 7. 1.] | [9. 5. 8.]
[7. 9. 8.] | [3. 5. 4.] | [6. 2. 1.]
------------------------------------
[1. 6. 9.] | [5. 8. 7.] | [4. 3. 2.]
[5. 7. 2.] | [4. 3. 9.] | [1. 8. 6.]
[8. 3. 4.] | [1. 2. 6.] | [5. 7. 9.]
------------------------------------
[3. 2. 6.] | [7. 1. 5.] | [8. 9. 4.]
[4. 8. 5.] | [9. 6. 3.] | [2. 1. 7.]
[9. 1. 7.] | [8. 4. 2.] | [3. 6. 5.]


------ Resuelto en:  26.1948161125 segundos ------

---------------- AC

In [43]:
btn,actn = execAll("hard.csv")
bt.append(btn)
act.append(actn)


---------------- Tablero a resolver ----------------


[0. 0. 2.] | [0. 1. 5.] | [3. 0. 0.]
[5. 0. 0.] | [9. 0. 6.] | [7. 0. 0.]
[0. 1. 0.] | [0. 0. 0.] | [6. 0. 0.]
------------------------------------
[0. 0. 0.] | [0. 5. 0.] | [4. 7. 1.]
[0. 0. 1.] | [7. 0. 0.] | [0. 0. 0.]
[0. 0. 0.] | [0. 0. 2.] | [0. 5. 0.]
------------------------------------
[0. 9. 0.] | [0. 2. 0.] | [0. 4. 0.]
[0. 0. 0.] | [0. 0. 0.] | [0. 0. 3.]
[4. 0. 0.] | [3. 0. 1.] | [0. 0. 0.]



---------------- BackTracking ---------------------


[7. 6. 2.] | [8. 1. 5.] | [3. 9. 4.]
[5. 8. 3.] | [9. 4. 6.] | [7. 1. 2.]
[9. 1. 4.] | [2. 7. 3.] | [6. 8. 5.]
------------------------------------
[2. 3. 9.] | [6. 5. 8.] | [4. 7. 1.]
[8. 5. 1.] | [7. 9. 4.] | [2. 3. 6.]
[6. 4. 7.] | [1. 3. 2.] | [8. 5. 9.]
------------------------------------
[3. 9. 6.] | [5. 2. 7.] | [1. 4. 8.]
[1. 7. 8.] | [4. 6. 9.] | [5. 2. 3.]
[4. 2. 5.] | [3. 8. 1.] | [9. 6. 7.]


------ Resuelto en:  43.442209959 segundos ------

---------------- AC3

In [ ]:
btn,actn = execAll("insane.csv")
bt.append(btn)
act.append(actn)


---------------- Tablero a resolver ----------------


[0. 0. 8.] | [0. 0. 4.] | [0. 7. 0.]
[0. 0. 0.] | [0. 1. 0.] | [0. 0. 3.]
[0. 0. 0.] | [0. 0. 0.] | [8. 0. 0.]
------------------------------------
[3. 0. 7.] | [0. 5. 0.] | [0. 0. 0.]
[0. 2. 0.] | [3. 6. 0.] | [0. 1. 0.]
[0. 0. 0.] | [0. 9. 0.] | [0. 0. 0.]
------------------------------------
[5. 6. 3.] | [0. 0. 0.] | [7. 0. 0.]
[0. 0. 0.] | [0. 0. 0.] | [0. 8. 0.]
[1. 0. 9.] | [5. 0. 0.] | [0. 0. 0.]



---------------- BackTracking ---------------------


In [ ]:
indexes = ["Test","Easy","Medium","Hard","Insane"]
dataset = pd.DataFrame({"BackTracking": bt,"AC3": act},index = indexes)
dataset    